In [8]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from tir_lib import *
from globals import *
import time
import importlib
import pandas as pd
import math
from tqdm import tqdm
import json
import sys
from utils import *

# MATH_HINTS = {
#     "complex calculation hint": "Wait, I can use Python to perform these complex calculations for this problem.```python\n",
#     "self reflection hint": "Wait, I can use Python to check if my approach is correct and refine it, if necessary.```python\n",
#     "general hint": "Wait, using python here may be a good idea.```python\n",
#     "alternative method hint": "Alternatively, I can use Python to explore an alternative method for solving this problem.```python\n",
#     "explore deeply hint": "Wait, I can explore deeply about this problem through python tools.```python\n",
#     "use python again hint": """Wait, I need to utilize Python code again to meticulously check to make sure I understand the question correctly as well as reasoning correctly.```python\n""",
# #     "check logic hint": "Maybe Python can assist in ensuring our logical deductions are sound.```python\n",
# #     "algebraic manipulation hint": "Wait, I can use Python to handle the algebraic manipulation and simplify the expression.```python\n",
# #     "solve equations hint": "Wait, I can use Python to solve this equation.```python\n",
# #     "series or sequences hint": "Wait, I can use Python to compute the terms of this series or sequence and identify patterns.```python\n",
# #     "validate assumption hint": "Wait, I can use Python to validate the assumptions I made in this calculation.```python\n",
# }

################## CONFIG #######
input_json = "wrongs_out_of_18k_heavily_checked.json"
target_TIR_json = "TIR_FULL_RESPONSES_ROUND_1_18k_DS_1.json"
not_worked_json = "AFTER_ROUND_1_RESPONSES_18k_DS_1.json"

def read_json_1(input_json):
    with open(input_json, "r") as f:
        data = json.load(f)
        return data
        
TIR_FULL_RESPONSES = []
AFTER_ROUND_RESPONSES = []
#################################
data_list = read_json_1(input_json)
N = len(data_list)
print(f"samples = {N}")
all_q = [item["problem"] for item in data_list]
all_r = [item["outputs"] for item in data_list]
all_a = [item["answer"] for item in data_list]

samples = 7554


In [9]:
#################################
def get_tir(q,r,a):
    length_response = get_length(r)
    b = True
    code_generated_freq = r.count("```python")
    output_generated_freq = r.count("```output")
    ########### IF CODE IS PRESENT ######################
    if code_generated_freq == output_generated_freq + 1:
#         resp = get_code_completion(q, r)
        new_r = r + "```"
        resp_with_output = add_output_at_end(new_r)
        added_response, real_tir = get_extra_completion(q, resp_with_output)
        full_response = resp_with_output + added_response
    elif code_generated_freq > 0:
            added_r = insert_hint(q,r)
            resp = get_code_completion(q, added_r)
            new_r = added_r + resp + "```"
            resp_with_output, real_tir = add_output_at_end(new_r)
            added_response = get_extra_completion(q, resp_with_output)
            full_response = resp_with_output + added_response
    ########### IF CODE IS NOT PRESENT ######################
    elif code_generated_freq == 0:
        choice = random.choices([1,0], weights = [0.1, 0.9], k = 1)[0]
        if choice or length_response < 3200:
            added_r = add_prompt_at_end(r)
            resp = get_code_completion(q, added_r)
            new_r = added_r + resp + "```"
            resp_with_output, real_tir = add_output_at_end(new_r)
            added_response = get_extra_completion(q, resp_with_output)
            full_response = resp_with_output + added_response
        if length_response > 3200 - 1:
            added_r = insert_hint(q,r)
            resp = get_code_completion(q, added_r)
            new_r = added_r + resp + "```"
            resp_with_output, real_tir = add_output_at_end(new_r)
            added_response = get_extra_completion(q, resp_with_output)
            full_response = resp_with_output + added_response
    else:
        raise Exception("some issue with code if - else") 
        
    gen_ans = extract_boxed_texts(full_response)
    if gen_ans is None:
        gen_ans = get_answer(full_response)
    if real_tir and (gen_ans == str(a) or int(verify_correctness(gen_ans, a))):
        TIR_FULL_RESPONSES.append({"q": q, "r":full_response, "a":a})
        print("TIR succeeded !")
    else:
        AFTER_ROUND_RESPONSES.append({"q": q, "r":full_response, "a":a})
        print("TIR failed...")
        b = False
    return b

In [10]:
len(TIR_FULL_RESPONSES)

0

In [11]:
target_TIR_json = "TIR_FULL_RESPONSES_ROUND_1_18k_DS_1.json"
not_worked_json = "AFTER_ROUND_1_RESPONSES_18k_DS_1.json"

In [ ]:
%%time
# math.ceil(len(data_list)/200)
for i in tqdm(range(math.ceil(len(data_list)/200))):
    start = 200*i
    end = 200*(i+1) 
    return_val = parallelize(get_tir, all_q[start:end] , all_r[start:end], all_a[start:end])
    print(return_val)

  0%|          | 0/38 [00:00<?, ?it/s]

TIR failed...
TIR succeeded !
TIR failed...
TIR succeeded !
TIR failed...
TIR succeeded !
TIR succeeded !
TIR failed...
TIR failed...
TIR succeeded !
TIR failed...
TIR failed...
TIR failed...
TIR failed...
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
api error occured!
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR succeeded !
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR succeeded !
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR failed...
TIR fa

  3%|▎         | 1/38 [03:05<1:54:13, 185.23s/it]

TIR failed...
[False, False, True, False, False, True, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, Fals

  5%|▌         | 2/38 [07:02<2:09:35, 215.98s/it]

TIR failed...
[False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, True, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, True, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, True, False, False,

In [ ]:
6

In [11]:
print(return_val)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, True, False, False, False, True, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, Fal

In [5]:
def save_unique(dict_list, name = "xx.json"):
    json_unique = []
    st = set()
    for item in dict_list:
        if item["q"] in st:
            pass
        else:
            st.add(item["q"])
            json_unique.append(item)
    print(f"final length: {len(json_unique)}")
    with open(name, "w") as f:
        json.dump(json_unique, f , indent = 4)
        
save_unique(TIR_FULL_RESPONSES, target_TIR_json)
save_unique(AFTER_ROUND_RESPONSES, not_worked_json)

final length: 606
final length: 6712
